In [2]:
# 数据处理
import pandas as pd
import numpy as np
import json
import csv
import random
import math
from itertools import product


# 网络分析与可视化
import networkx as nx
from networkx.readwrite import json_graph
from pyvis import network as net
import matplotlib.pyplot as plt
import powerlaw # Power laws are probability distributions with the form:p(x)∝x−α

In [7]:
with open('02_Graph/MasterConceptNetwork.json') as f:
    js_graph = json.load(f)
    G_MasterConceptNetwork = json_graph.node_link_graph(js_graph)

df_concept_info = pd.read_csv('00_Raw Data/CCFD_concept_info.csv')

In [9]:
# 查看全部节点在某个属性的所有选项
def NodeAttributeValueList(G,attribute):
    return set(np.array([G.nodes[n][attribute] for n in G.nodes]).flatten())


In [10]:
# maincategory 和 subcategory 的列表
maincategorys = NodeAttributeValueList(G_MasterConceptNetwork,'maincategory')
subcategorys = NodeAttributeValueList(G_MasterConceptNetwork,'subcategory')

In [23]:
df_concept_info[df_concept_info['subcategory']=="tool"].上级类别.unique()[0]

'人造物'

In [24]:
# 抽取subcategorys子图间的关联网络 Graph Summary for subcategorys
G_Summary = nx.Graph()
G_Summary.add_nodes_from(subcategorys)
G_Summary.add_edges_from((a,b) for a,b in product(subcategorys,subcategorys) if a!=b)

# add node count into each node
for n,v in G_Summary.nodes(data=True):
    v['count'] = len(df_concept_info[df_concept_info['下级类别']==n])
    v['subcategory'] = n
    # print(n)
    v['maincategory'] = df_concept_info[df_concept_info['下级类别']==n].上级类别.unique()[0]


# add edge weight into each edge
for u,v,e in G_Summary.edges(data=True):
    e['weight'] = 0

for u,v,e in G_MasterConceptNetwork.edges(data=True):
    u_subcategory = G_MasterConceptNetwork.nodes[u]['subcategory']
    v_subcategory = G_MasterConceptNetwork.nodes[v]['subcategory']
    if  u_subcategory!=v_subcategory:
        G_Summary[u_subcategory][v_subcategory]['weight']+=e['weight']

# 导出 for G6
with open('02_Graph/MasterConceptNetwork_Summary_G6.json','w+') as f:
    f.write(json.dumps(nx.node_link_data(G_Summary), ensure_ascii=False).replace('links','edges'))

# with open('02_Graph/Wenyue/ConceptNetwork_Summary.json','w+') as f:
#     f.write(json.dumps(nx.node_link_data(G_Summary), ensure_ascii=False))

# 导出 for Gephi
# nx.write_gexf(G_Summary, "02_Graph/ConceptNetwork_Summary_CCFD-Similarity-Matrix_Weight-0.52.gexf")